In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
import math

# Загрузка данных и их обработка

In [25]:
iris = datasets.load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data['target'] = iris.target

data = data.loc[(data["target"] == 1) | (data["target"] == 2)] # выбор нужных классов ириса
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 50 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  100 non-null    float64
 1   sepal width (cm)   100 non-null    float64
 2   petal length (cm)  100 non-null    float64
 3   petal width (cm)   100 non-null    float64
 4   target             100 non-null    int32  
dtypes: float64(4), int32(1)
memory usage: 4.3 KB


In [26]:
data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [27]:
X = data.drop(["target"], axis=1)
Y = data["target"].map({2: 1, 1: 0})
Y.value_counts()

0    50
1    50
Name: target, dtype: int64

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Реализация класса логистической регрессии

In [44]:
class MyLogisticRegression:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.weight = None
        self.bias = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def cross_entropy(self, h0, y, m):
        h0 = np.clip(h0, 0.0001, 0.9999)
        return np.sum(-y * np.log(h0) - (1 - y) * np.log(1 - h0))/m

    def fit_gradient(self, X_train, Y_train, num_iterations=10000):
        X_train = X_train.values
        Y_train = Y_train.values
        X_train = X_train.T
        Y_train = Y_train.reshape(1, X_train.shape[1])
        m = X_train.shape[1]
        n = X_train.shape[0]
        self.weight = np.zeros((n,1))
        self.bias = 0
        cost_list = []
        for i in range(num_iterations):
            z = np.dot(self.weight.T, X_train) + self.bias
            h0 = self.sigmoid(z)
            update_weight = np.dot(h0 - Y_train, X_train.T) / m
            update_bias = np.sum(h0 - Y_train) / m
            self.weight = self.weight - self.learning_rate * update_weight.T
            self.bias = self.bias - self.learning_rate * update_bias
            if i % 1000 == 0:
                cost = self.cross_entropy(h0, Y_train, m)
                cost_list.append(cost)
                print(f"После {i} итерации значение функции стоимости = {cost}")
                print(self.weight, self.bias)
        return cost_list

    def fit_rmsprop(self, X_train, Y_train, beta=0.9, eps=1e-8, num_iterations=1000):
        X_train = X_train.values
        Y_train = Y_train.values
        X_train = X_train.T
        Y_train = Y_train.reshape(1, X_train.shape[1])
        m = X_train.shape[1]
        n = X_train.shape[0]
        self.weight = np.random.randn(1, n)
        self.bias = 0
        self.v_dw = np.zeros_like(self.weight)
        self.v_db = 0
        cost_list = []
        for i in range(num_iterations):
            z = np.dot(self.weight, X_train) + self.bias
            h0 = self.sigmoid(z)
            dw = np.dot(h0 - Y_train, X_train.T) / m
            db = np.sum(h0 - Y_train) / m
            self.v_dw = beta * self.v_dw + (1 - beta) * dw**2
            self.v_db = beta * self.v_db + (1 - beta) * db**2
            self.weight -= self.learning_rate * dw / (np.sqrt(self.v_dw) + eps)
            self.bias -= self.learning_rate * db / (np.sqrt(self.v_db) + eps)
            if i % 1000 == 0:
                cost = self.cross_entropy(h0, Y_train, m)
                cost_list.append(cost)
                print(f"После {i} итерации значение функции стоимости = {cost}")
                print("Weight: ", self.weight)
                print("Bias: ", self.bias)
                print(h0)
        self.weight = self.weight.T
        return cost_list
    
    
    def predict(self, X_pred):
        X_pred = X_pred.values
        X_pred = X_pred.T
        z = np.dot(self.weight.T, X_pred) + self.bias
        h0 = self.sigmoid(z)
        print("H0: ", h0)
        Y_pred = np.zeros((1, X_pred.shape[1]))
        for i in range(h0.shape[1]):
            if h0[0, i] <= 0.5:
                Y_pred[0, i] = 0
            else:
                Y_pred[0, i] = 1
        Y_pred = Y_pred.flatten()
        Y_pred = pd.DataFrame({"Result": Y_pred})
        return Y_pred

In [45]:
from sklearn.metrics import accuracy_score

model = MyLogisticRegression()
model.fit_gradient(X_train, Y_train)
Y_pred = model.predict(X_test)
print(accuracy_score(Y_pred, Y_test))
print(Y_test.value_counts())

После 0 итерации значение функции стоимости = 0.6931471805599454
[[0.0034    ]
 [0.00127143]
 [0.00475   ]
 [0.00231429]] 0.00028571428571428574
После 1000 итерации значение функции стоимости = 0.40869143290929394
[[-0.67915507]
 [-0.58448824]
 [ 1.01759466]
 [ 0.86969162]] -0.31433885063613604
После 2000 итерации значение функции стоимости = 0.2991430927752731
[[-1.09843603]
 [-0.98651859]
 [ 1.63920377]
 [ 1.42600597]] -0.5344479715681404
После 3000 итерации значение функции стоимости = 0.24227154245542323
[[-1.38432854]
 [-1.29469355]
 [ 2.07970348]
 [ 1.83148578]] -0.7067284798733726
После 4000 итерации значение функции стоимости = 0.20733484763717766
[[-1.59679019]
 [-1.54858962]
 [ 2.41925464]
 [ 2.15098003]] -0.8514609642739828
После 5000 итерации значение функции стоимости = 0.18349465725560563
[[-1.76366813]
 [-1.76696394]
 [ 2.69535658]
 [ 2.4153782 ]] -0.9782450276660934
После 6000 итерации значение функции стоимости = 0.16604476748336885
[[-1.89979822]
 [-1.96010663]
 [ 2.9

In [46]:
model2 = MyLogisticRegression()
model2.fit_rmsprop(X_train, Y_train)
Y_pred = model2.predict(X_test)
print(Y_pred)
print("---------")
print(Y_test)
print(accuracy_score(Y_pred, Y_test))

После 0 итерации значение функции стоимости = 1.6401231062112547
Weight:  [[ 1.08086355 -1.55820933  0.32943241 -0.30644855]]
Bias:  -0.03162277435123163
[[0.95639932 0.9750491  0.99615344 0.96953761 0.98490165 0.96558377
  0.98436552 0.97114882 0.93520581 0.9459701  0.96044796 0.93340292
  0.93726412 0.98400392 0.9853992  0.97123094 0.99141158 0.98141311
  0.98779726 0.97518718 0.97992646 0.9757425  0.97169868 0.94121122
  0.98549841 0.98648363 0.96188058 0.96642357 0.98211877 0.98682668
  0.94225619 0.96546394 0.98434066 0.95110451 0.96923695 0.98984487
  0.95577487 0.99843171 0.98774575 0.98164336 0.99384637 0.97018585
  0.99470996 0.93327194 0.95367405 0.99582314 0.99180879 0.96798343
  0.97783601 0.98252856 0.97769376 0.98850401 0.99124107 0.96646043
  0.96915187 0.99240005 0.97336859 0.98662192 0.97974504 0.97973945
  0.98452631 0.98009743 0.96009753 0.98611224 0.94867902 0.97436467
  0.95988256 0.93965006 0.97466358 0.97466358]]
H0:  [[0.57736557 0.98710888 0.99558893 0.0023155 

In [21]:
class RMSprop:
    def __init__(self, learning_rate=0.001, beta=0.9, eps=1e-8):
        self.learning_rate = learning_rate
        self.beta = beta
        self.eps = eps
        self.v_dw = None
        self.v_db = None
        self.weight = None
        self.bias = None

    def fit(self, X_train, Y_train, num_iterations):
        m = X_train.shape[1]
        n = X_train.shape[0]
        
        # Инициализация параметров и скоростей
        self.weight = np.random.randn(1, n)
        self.bias = 0
        self.v_dw = np.zeros_like(self.weight)
        self.v_db = 0

        for i in range(num_iterations):
            # Выполнение шагов обновления весов и смещения с использованием RMSprop
            z = np.dot(self.weight, X_train) + self.bias
            h0 = self.sigmoid(z)
            dw = np.dot(h0 - Y_train, X_train.T) / m
            db = np.sum(h0 - Y_train) / m
            self.v_dw = self.beta * self.v_dw + (1 - self.beta) * dw**2
            self.v_db = self.beta * self.v_db + (1 - self.beta) * db**2
            self.weight -= self.learning_rate * dw / (np.sqrt(self.v_dw) + self.eps)
            self.bias -= self.learning_rate * db / (np.sqrt(self.v_db) + self.eps)

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def predict(self, X_pred):
        z = np.dot(self.weight, X_pred) + self.bias
        h0 = self.sigmoid(z)
        print("H0: ", h0)
        Y_pred = np.zeros((1, X_pred.shape[1]))
        for i in range(h0.shape[1]):
            if h0[0, i] <= 0.5:
                Y_pred[0, i] = 0
            else:
                Y_pred[0, i] = 1
        Y_pred = Y_pred.flatten()
        Y_pred = pd.DataFrame({"Result": Y_pred})
        return Y_pred

# Генерация синтетических данных для демонстрации работы класса
X_train = np.random.rand(4, 1000)
Y_train = np.random.randint(2, size=(1, 1000))

print(X_train.shape, Y_train.shape)
# Создание экземпляра класса RMSprop и обучение модели
model = RMSprop()
model.fit(X_train, Y_train, num_iterations=1000)
Y_predict = model.predict(X_train)
print(accuracy_score(Y_predict, Y_train))

(4, 1000) (1, 1000)
H0:  [[0.52315703 0.44678912 0.47993558 0.45513833 0.50163476 0.48427842
  0.46893673 0.43969387 0.50829021 0.50232544 0.50360407 0.52403908
  0.50303783 0.46967056 0.51073196 0.46498517 0.49135884 0.47194119
  0.53340667 0.48959854 0.46153946 0.47700751 0.45347268 0.5294167
  0.50486673 0.49014116 0.48631251 0.44895577 0.52013029 0.44912237
  0.50606198 0.48956249 0.49762078 0.44660809 0.49981698 0.52089109
  0.50597762 0.4711949  0.48128517 0.46867232 0.50471649 0.52712184
  0.42135277 0.51922675 0.50093134 0.4507493  0.44307202 0.52815018
  0.53089734 0.47744649 0.48301544 0.45248158 0.50127065 0.474955
  0.48474491 0.46774936 0.51211023 0.46968645 0.4577545  0.52892194
  0.5041143  0.46218609 0.48623821 0.54659143 0.46210093 0.52849775
  0.47141433 0.50818216 0.50851918 0.52570497 0.5177481  0.48527631
  0.47616012 0.49710558 0.43419248 0.47453991 0.53208174 0.55384854
  0.47517555 0.49994051 0.51710252 0.52828976 0.46848683 0.52753827
  0.46597147 0.47284714 0.

ValueError: Found input variables with inconsistent numbers of samples: [1000, 1]